## BLE RSSI Dataset for Indoor localization

The dataset was created using the RSSI readings of an array of 13 ibeacons in the first floor of Waldo Library, Western Michigan University. Data was collected using iPhone 6S. The dataset contains two sub-datasets: a labeled dataset (1420 instances) and an unlabeled dataset (5191 instances). The recording was performed during the operational hours of the library. For the labeled dataset, the input data contains the location (label column), a timestamp, followed by RSSI readings of 13 iBeacons. RSSI measurements are negative values. Bigger RSSI values indicate closer proximity to a given iBeacon (e.g., RSSI of -65 represent a closer distance to a given iBeacon compared to RSSI of -85). For out-of-range iBeacons, the RSSI is indicated by -200. The locations related to RSSI readings are combined in one column consisting a letter for the column and a number for the row of the position. The following figure depicts the layout of the iBeacons as well as the arrange of locations.

![alt text](pictures/iBeacon_Layout.jpg "Title")

## Install Libraries

In [62]:
! pip install pandas sklearn --user

## Import Libraries

In [63]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import os
from sklearn.preprocessing import OneHotEncoder

## Declare Variables

In [64]:
BLE_RSSI = pd.read_csv('iBeacon_RSSI_Labeled.csv') #Labeled dataset

# Configure model options
TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "blerssi/")
TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "blerssi/")
TF_MODEL_TYPE = os.getenv("TF_MODEL_TYPE", "DNN")
TF_TRAIN_STEPS = int(os.getenv("TF_TRAIN_STEPS", 5000))
TF_BATCH_SIZE = int(os.getenv("TF_BATCH_SIZE", 50))
TF_LEARNING_RATE = float(os.getenv("TF_LEARNING_RATE", 0.001))


# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
LABEL = [COLUMNS[0]]

INPUT_FEATURE = 'x'
NUM_CLASSES = 3

## BLERSSI Input Dataset

### Attribute Information

    location: The location of receiving RSSIs from ibeacons b3001 to b3013; 
              symbolic values showing the column and row of the location on the map (e.g., A01 stands for column A, row 1).
    date: Datetime in the format of ‘d-m-yyyy hh:mm:ss’
    b3001 - b3013: RSSI readings corresponding to the iBeacons; numeric, integers only.

In [65]:
BLE_RSSI.head(10)

,location,date,b3001,b3002,b3003,b3004,b3005,b3006,b3007,b3008,b3009,b3010,b3011,b3012,b3013
0,O02,10-18-2016 11:15:21,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
1,P01,10-18-2016 11:15:19,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
2,P01,10-18-2016 11:15:17,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
3,P01,10-18-2016 11:15:15,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
4,P01,10-18-2016 11:15:13,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
5,P01,10-18-2016 11:15:11,-200,-200,-82,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
6,P01,10-18-2016 11:15:09,-200,-200,-80,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
7,P02,10-18-2016 11:15:07,-200,-200,-86,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
8,R01,10-18-2016 11:15:05,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200
9,R01,10-18-2016 11:15:03,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200


## Definition of Serving Input Receiver Function

In [66]:
def serving_input_receiver_fn1():
    """
    This is used to define inputs to serve the model.
    :return: ServingInputReciever
    """
    receiver_tensors = {
        'b3001': tf.placeholder(tf.float32, [None, 1]),
        'b3002': tf.placeholder(tf.float32, [None, 1]),
        'b3003': tf.placeholder(tf.float32, [None, 1]),
        'b3004': tf.placeholder(tf.float32, [None, 1]),
        'b3005': tf.placeholder(tf.float32, [None, 1]),
        'b3006': tf.placeholder(tf.float32, [None, 1]),
        'b3007': tf.placeholder(tf.float32, [None, 1]),
        'b3008': tf.placeholder(tf.float32, [None, 1]),
        'b3009': tf.placeholder(tf.float32, [None, 1]),
        'b3010': tf.placeholder(tf.float32, [None, 1]),
        'b3011': tf.placeholder(tf.float32, [None, 1]),
        'b3012': tf.placeholder(tf.float32, [None, 1]),
        'b3013': tf.placeholder(tf.float32, [None, 1])

    }

    # Convert give inputs to adjust to the model.
    features = {
         INPUT_FEATURE: tf.concat([
            receiver_tensors['b3001'],
            receiver_tensors['b3002'],
            receiver_tensors['b3003'],
            receiver_tensors['b3004'],
            receiver_tensors['b3005'],
            receiver_tensors['b3006'],
            receiver_tensors['b3007'],
            receiver_tensors['b3008'],
            receiver_tensors['b3009'],
            receiver_tensors['b3010'],
            receiver_tensors['b3011'],
            receiver_tensors['b3012'],
            receiver_tensors['b3013'],

        ], axis=1)
    }
    return tf.estimator.export.ServingInputReceiver(receiver_tensors=receiver_tensors,
                                                    features=features)


## Train and Save BLE RSSI Model

In [67]:
def main(unused_args):
  tf.logging.set_verbosity(tf.logging.INFO)

  df_full = pd.read_csv('iBeacon_RSSI_Labeled.csv') #Labeled dataset

  # Input Data Preprocessing 
  df_full = df_full.drop(['date'],axis = 1)
  df_full[FEATURES] = (df_full[FEATURES])/(-200)

  #Output Data Preprocessing
  dict = {'O02': 0,'P01': 1,'P02': 2,'R01': 3,'R02': 4,'S01': 5,'S02': 6,'T01': 7,'U02': 8,'U01': 9,'J03': 10,'K03': 11,'L03': 12,'M03': 13,'N03': 14,'O03': 15,'P03': 16,'Q03': 17,'R03': 18,'S03': 19,'T03': 20,'U03': 21,'U04': 22,'T04': 23,'S04': 24,'R04': 25,'Q04': 26,'P04': 27,'O04': 28,'N04': 29,'M04': 30,'L04': 31,'K04': 32,'J04': 33,'I04': 34,'I05': 35,'J05': 36,'K05': 37,'L05': 38,'M05': 39,'N05': 40,'O05': 41,'P05': 42,'Q05': 43,'R05': 44,'S05': 45,'T05': 46,'U05': 47,'S06': 48,'R06': 49,'Q06': 50,'P06': 51,'O06': 52,'N06': 53,'M06': 54,'L06': 55,'K06': 56,'J06': 57,'I06': 58,'F08': 59,'J02': 60,'J07': 61,'I07': 62,'I10': 63,'J10': 64,'D15': 65,'E15': 66,'G15': 67,'J15': 68,'L15': 69,'R15': 70,'T15': 71,'W15': 72,'I08': 73,'I03': 74,'J08': 75,'I01': 76,'I02': 77,'J01': 78,'K01': 79,'K02': 80,'L01': 81,'L02': 82,'M01': 83,'M02': 84,'N01': 85,'N02': 86,'O01': 87,'I09': 88,'D14': 89,'D13': 90,'K07': 91,'K08': 92,'N15': 93,'P15': 94,'I15': 95,'S15': 96,'U15': 97,'V15': 98,'S07': 99,'S08': 100,'L09': 101,'L08': 102,'Q02': 103,'Q01': 104}
  df_full['location'] = df_full['location'].map(dict)
  df_train=df_full.sample(frac=0.8,random_state=200)
  df_valid=df_full.drop(df_train.index)

  location_counts = BLE_RSSI.location.value_counts()
  train_X = np.asarray(df_train[FEATURES])
  train_y = np.asarray(df_train['location'])
  test_X = np.asarray(df_valid[FEATURES])
  test_y = np.asarray(df_valid['location'])
  feature_columns = [
        tf.feature_column.numeric_column(INPUT_FEATURE, shape=[13])
  ]
  # Train Input Function
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {INPUT_FEATURE: train_X},
    y = train_y,
    batch_size = 12,
    num_epochs = 100,
    shuffle = False,

  )

  # Test Input Function
  test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {INPUT_FEATURE: test_X},
    y = test_y,
    batch_size = 12,
    num_epochs = 100,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )


  config = tf.estimator.RunConfig(model_dir=TF_MODEL_DIR, save_summary_steps=100, save_checkpoints_steps=1000)

  # Build 3 layer DNN classifier

  model = tf.estimator.DNNClassifier(hidden_units = [13,65,110],
                     feature_columns = feature_columns,
                     model_dir = TF_MODEL_DIR,
                     n_classes=105, config=config
                   )

  export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn1)

  train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, 
                                        max_steps=TF_TRAIN_STEPS)

  eval_spec = tf.estimator.EvalSpec(input_fn=test_input_fn,
                                      steps=1,
                                      exporters=export_final,
                                      throttle_secs=1,
                                      start_delay_secs=1)

  # Train and Evaluate the model

  tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

if __name__ == "__main__":
    tf.app.run() 

INFO:tensorflow:Using config: {'_model_dir': 'blerssi/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1f50536748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0415 10:06:34.601049 139775581214528 estimator.py:212] Using config: {'_model_dir': 'blerssi/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1f50536748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Not using Distribute Coordinator.


I0415 10:06:34.602662 139775581214528 estimator_training.py:186] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


...

I0415 10:06:47.805230 139775581214528 builder_impl.py:425] SavedModel written to: blerssi/export/blerssi/temp-b'1586945207'/saved_model.pb


INFO:tensorflow:Loss for final step: 33.769516.


I0415 10:06:47.882945 139775581214528 estimator.py:371] Loss for final step: 33.769516.


SystemExit: 

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Update  storageUri in blerssi_kfserving.yaml with pvc-name

In [4]:
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
! sed -i "s/nfs/$pvc/g" blerssi_kfserving.yaml
! cat blerssi_kfserving.yaml

apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "blerssi-model"
  namespace: anonymous
spec:
  default:
    predictor:
      tensorflow:
        storageUri: "pvc://workspace-multi-gpu1/blerssi/export/blerssi"

## Serving BLE RSSI Model using kubeflow kfserving

In [5]:
!kubectl apply -f blerssi_kfserving.yaml

inferenceservice.serving.kubeflow.org/blerssi-model created


In [73]:
!kubectl get inferenceservices -n anonymous

NAME            URL                                                                  READY   DEFAULT TRAFFIC   CANARY TRAFFIC   AGE
blerssi-model   http://blerssi-model.anonymous.example.com/v1/models/blerssi-model   True    100                                93s


#### Note:
Wait for inference service READY="True"

## Data pre-process for prediction

In [75]:
  df_full = pd.read_csv('iBeacon_RSSI_Unlabeled_truncated.csv') #Labeled dataset
  # Input Data Preprocessing 
  df_full = df_full.drop(['date'],axis = 1)
  df_full = df_full.drop(['location'],axis = 1)
  df_full[FEATURES] = (df_full[FEATURES])/(-200)
  print(df_full.head(5))

   b3001  b3002  b3003  b3004  b3005  b3006  b3007  b3008  b3009  b3010  \
0    1.0    1.0  0.400    1.0  0.385  0.280  0.405    1.0    1.0    1.0   
1    1.0    1.0  0.400    1.0  0.390  0.280  1.000    1.0    1.0    1.0   
2    1.0    1.0  0.405    1.0  0.370  0.315  1.000    1.0    1.0    1.0   
3    1.0    1.0  1.000    1.0  0.375  0.280  1.000    1.0    1.0    1.0   
4    1.0    1.0  1.000    1.0  0.410  0.280  1.000    1.0    1.0    1.0   

   b3011  b3012  b3013  
0    1.0    1.0    1.0  
1    1.0    1.0    1.0  
2    1.0    1.0    1.0  
3    1.0    1.0    1.0  
4    1.0    1.0    1.0  


## Predict data from serving after setting INGRESS_IP
### Note - Use one of preprocessed row values from previous cell in request data instances

In [76]:
!curl -v -H "Host: blerssi-model.anonymous.example.com" http://INGRESS_IP:31380/v1/models/blerssi-model:predict -d '{"signature_name":"predict","instances":[{"b3001":[1.0] , "b3002":[1.0] , "b3003":[0.4], "b3004":[1.0] , "b3005":[0.385] , "b3006":[0.280] , "b3007":[0.405] , "b3008":[1.0] , "b3009":[1.0] , "b3010":[1.0] , "b3011":[1.0] , "b3012":[1.0] , "b3013":[1.0]}]}'

*   Trying INGRESS_IP...
* TCP_NODELAY set
* Connected to INGRESS_IP (INGRESS_IP) port 31380 (#0)
> POST /v1/models/blerssi-model:predict HTTP/1.1
> Host: blerssi-model.anonymous.example.com
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Length: 255
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 255 out of 255 bytes
< HTTP/1.1 200 OK
< content-length: 4243
< content-type: application/json
< date: Wed, 15 Apr 2020 10:10:14 GMT
< x-envoy-upstream-service-time: 8810
< server: istio-envoy
< 
{
    "predictions": [
        {
            "class_ids": [14],
            "classes": ["14"],
            "all_class_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 8

## Delete kfserving model & Clean up of stored models

In [6]:
!kubectl delete -f blerssi_kfserving.yaml
!rm -rf /mnt/blerssi
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
! sed -i "s/$pvc/nfs/g" blerssi_kfserving.yaml
! cat blerssi_kfserving.yaml

inferenceservice.serving.kubeflow.org "blerssi-model" deleted
apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "blerssi-model"
  namespace: anonymous
spec:
  default:
    predictor:
      tensorflow:
        storageUri: "pvc://nfs1/blerssi/export/blerssi"